# JavaScript pragmatics

## Some recurring choices

Programing style considerations and best practices issues, not exclusively related to ES5/6 spec support.

### Variable naming

(Refs: [1](http://stackoverflow.com/questions/101070/what-is-an-ideal-variable-naming-convention-for-loop-variables); [2](http://stackoverflow.com/questions/2886320/int-i-vs-int-index-etc-which-one-is-better); [3](http://programmers.stackexchange.com/questions/71710/using-single-characters-for-variable-names-in-loops-exceptions); [4](http://stackoverflow.com/questions/2076146/do-you-use-1-3-letters-variables-everywhere))

* The wider the scope of a variable, the more descriptive a variable name should be
* Single-letter variable names: `i`, `j`, `k` as loop iterators are widely considered acceptable, as well as `e` for event
    * `l` is hard to read (can be mistaken for `1`), so don't use
* More broadly: `i, j, k, m, n` for integers and `x, y, z` for non-integers
* Three/four-letter variable names: `val`; `num`, `int`, `sum`; `str`, `char`; `arr`, `elt`, `obj`, `prop`; `out`, `err`
* Use prefix `n` to indicate number of objects: `nFiles`

### Function naming

Describe what the function does: `getMin` gets a minimum of values passed in as parameters.

I struggle with context specificity. If a parameter is an array, is `getArrayMin` better?

Using function names in function expressions if you want a function name rather than a binding to show up in a stack trace. E.g.: `window.onload = function init() {...};` rather than `window.onload = function() {...};` This is recommended in Braithwaite, *JavaScript Allongé* (and the example given is also an event listener: `someBackboneView.on('click', function clickHandler() {...};`). In the section **Functions** on this page, there's an example of naming an IIFE so it can call itself recursively.

### Iterating over arrays

Using `for..in` where array length and index order are unimportant; using a `for` loop or ES5 `forEach()` otherwise.

`forEach()` doesn't require use of bracket notation:

In [6]:
var arr = ['foo', 'bar'];

for (var elt in arr) console.log(arr[elt] + 'baz');

arr.forEach(function(elt) {console.log(elt + 'baz');});

foobaz
barbaz
foobaz
barbaz


undefined

## Primitives

### Validation

Very basic validation, rejecting null, undefined, and empty string:

In [48]:
function isTruthy(value) {
  return (!value) ? false : true; 
}

console.log(isTruthy(null));      // => false
console.log(isTruthy(""));        // => false
console.log(isTruthy(undefined)); // => false

console.log(isTruthy(1));         // => true
console.log(isTruthy("foo"));     // => true

false
false
false
true
true


undefined

However, this approach is foiled by the falsiness of zero, which means that we cannot use it to *validate* a numeric value of zero, if we want to do that:

In [11]:
function isTruthy(value) {
  return (!value) ? false : true; 
}

console.log(isTruthy(0));         // => false

false


undefined

A second condition must be added to handle the falsiness of zero:

In [16]:
function isTruthy(value) {
  return (!value || value === 0) ? false : true; 
}

console.log(isTruthy(null));      // => false
console.log(isTruthy(""));        // => false
console.log(isTruthy(undefined)); // => false

console.log(isTruthy(1));         // => true
console.log(isTruthy("foo"));     // => true

console.log(isTruthy(0));         // => false

false
false
false
true
true
false


undefined

### Testing for values you want to reject

When excluding values, is it more meaningful to use negation expressions than to use equality tests? For example, when expecting an array of values, testing for null, undefined or an empty array:

In [ ]:
(!excludeMe || !excludeMe.length)

The equivalent using equality tests:

In [ ]:
(excludeMe === null || excludeMe === undefined || excludeMe.length === 0)

#### Zero is falsey

The results of the evaluations of arithmetic expressions are either truthy or falsey:

In [30]:
console.log((1 + 1 || 2 + 2)); // => 2: (truthy || truthy)
console.log((1 + 1 && 2 + 2)); // => 4: (truthy && truthy)

console.log((1 + 1 || 2 - 2)); // => 2: (truthy || falsey)
console.log((1 + 1 && 2 - 2)); // => 0: (truthy && falsey)

console.log((1 - 1 || 2 + 2)); // => 4: (falsey || truthy)
console.log((1 - 1 && 2 + 2)); // => 0: (falsey && truthy)

console.log((1 - 1 || 2 - 2)); // => 0: (falsey || falsey)
console.log((1 - 1 && 2 - 2)); // => 0: (falsey && falsey)

2
4
2
0
4
0
0
0


undefined

We can return those properties using the logical negation (`!`) operator:

In [1]:
console.log(!(1 - 1 || 2 + 2)); // => false
console.log(!(1 - 1 && 2 + 2)); // => true

console.log(!(1 - 1 || 2 - 2)); // => true
console.log(!(1 - 1 && 2 - 2)); // => true

false
true
true
true


undefined

In the example below, `n % x` and `n % y` are each evaluated by `isNaN()`, then each result is evaluated for truthiness using the `!` operator. If the first evaluates to true (that is, if it is a number) **and** the second also evaluates to true (that is, if it is also a number), **and** both expressions in `(n % x || n % y)` evaluate to zero, then **because zero is a falsey value**, the logical negation of `n % x || n % y` is true, and we return `true && true && true`.

This handles **all** of the following cases:
* Values that are not numbers (including `null` and `undefined`)
* Division by zero (`n % 2` is evaluated as `NaN`)
* Fewer than three arguments, because an omitted argument will be `undefined`. (Any more than the three arguments specified as function parameters will be ignored, as usual in JavaScript).

In [18]:
function isDivisible(n, x, y) {
    // Zero is falsey
    return !isNaN(n % x)
           && !isNaN(n % y)
           && !(n % x || n % y);
}

// Tests
function terseTest(exp) {
    return exp ? 'success' : 'FAILURE';
}

console.log(
    terseTest(isDivisible(0, 1, 2)     === true),
    terseTest(isDivisible(3, 1, 3)     === true),
    terseTest(isDivisible(12, 7, 5)    === false),
    terseTest(isDivisible(0, 0, 0)     === false),
    terseTest(isDivisible(1, 1, 0)     === false),
    terseTest(isDivisible(1, 1, 0, 3)  === false), // Excess of three arguments
    terseTest(isDivisible(1)           === false), // Only one argument
    terseTest(isDivisible('foo', 2, 2) === false),
    terseTest(isDivisible(1, 2, '')    === false),
    terseTest(isDivisible(1, 2, undefined) === false),
    terseTest(isDivisible(1, 2, null)  === false)
);

success success success success success success success success success success success


undefined

### Generating id numbers using `Date()`

In [1]:
var id = Date.now(); // milliseconds
console.log(id);

id = Math.floor(Date.now() / 1000); // seconds
console.log(id);

1434996776536
1434996776


undefined

## Variables

### Using `void()`

Need an example here.

### Using `typeof`

Although `typeof` returns 'undefined' for **both *undefined* and *undeclared* variables**, it is the only way to test for an undeclared variable without throwing a `ReferenceError`:

In [4]:
// throws ReferenceError
myUndeclaredVariable 
    ? console.log('SUCCESS: myUndeclaredVariable was declared')
    : console.log('FAILURE: myUndeclaredVariable was not declared');

ReferenceError: evalmachine.<anonymous>:2
myUndeclaredVariable 
^
myUndeclaredVariable is not defined

In [6]:
/* At least returns 'undefined', though doesn't distinguish between 
   variables that haven't been declared at all, and variables that have 
   been declared but not initalized */
typeof myUndeclaredVariable; // => 'undefined'

'undefined'

Test to see if a variable is defined, and if it’s undefined, set it:

In [1]:
var myString = "foo";
myString = myString || "bar";

console.log(myString); // => "foo"

foo


undefined

In [2]:
var myString = "";
myString = myString || "bar";

console.log(myString); // => "bar"

bar


undefined

Use the above to test for omitted optional function parameters (see [JavaScript Function Parameters](http://www.w3schools.com/js/js_function_parameters.asp)):

In [3]:
function myFunction(x, y) {
    y = y || 0;
    return [x, y];
}
console.log(myFunction(1));    // => [1, 0]
console.log(myFunction(1, 2)); // => [1, 2]

[ 1, 0 ]
[ 1, 2 ]


undefined

The above approaches are better than using an expression like `if (!value) ...`, because of the falsiness of zero. Here, that expression works as intended, assigning the value `1` to the omitted function parameter `y`:

In [4]:
function myFunction(x, y) {
    if (!y) y = 1;
    console.log(x, y);
}

myFunction(0); // => "0 1"

0 1


undefined

Here, however, it does not work as intended. Because zero is falsey, `!0` returns `true`, and we cannot set the `y` parameter to the value `0`:

In [10]:
if (0) console.log('Zero is truthy');  // => ""
if (!0) console.log('Zero is falsey'); // => "Zero is falsey"

function myFunction(x, y) {
    if (!y) y = 1;                     // `(!0)` will return `true` here
    console.log(x, y);
}

// We want to set `y` to `0`, but it gets set to `1`
myFunction(1, 0); // => "1 1"

Zero is falsey
1 1


undefined

## Strings

String validation using a regexp and the RegExp prototype's `test()` method:

In [22]:
console.log(/\d{5}/.test('12345')); // => true

true


undefined

Counting the occurrence of specific characters in a string. The String prototype's `.match()` method returns an array of matches, so we can use that array's `length` property:

In [53]:
console.log('foo'.match(/o/g));        // => [o, o]
console.log('foo'.match(/o/g).length); // => 2

[ 'o', 'o' ]
2


undefined

Use the same technique to count words in a string:

In [62]:
var str = 'why are you and why am I and why are we and why are they';
console.log(str.match(/\bwhy\b/g).length); // => 4

4


undefined

Escaping special characters:

In [31]:
console.log('?+*'.replace(/[-[\]{}()*+?.,\\^$|#\s]/g, '\\$&'));

\?\+\*


undefined

Stripping comments from lines, using a bracket expression to prevent a comment delimiter like `$` from being interpreted as a regexp operator:

In [36]:
var delims = '#!$';
var re = new RegExp('\\s*[' + delims + '].*$', 'gm');

// => 'foo\nfoo\nfoo'
console.log('foo # bar\nfoo ! bar\nfoo # bar'.replace(re, ''));

foo
foo
foo


undefined

Title casing: use the String prototype's `charAt()` method to get a single character, and the `substring()` or `slice()` methods, passing only the starting index, to get the remaining portion of a string:

In [17]:
var str = 'foo';
console.log(str.charAt(0).toUpperCase() + str.substring(1)); // => 'Foo'
console.log(str.charAt(0).toUpperCase() + str.slice(1));     // => 'Foo'

Foo
Foo


undefined

Replace (here, leading and trailing spaces) using regexp:

In [5]:
var myString = "   I'm a string   ";

// => "I'm a string without leading or trailing spaces"
console.log(myString.replace(/^\s+|\s+$/g, "") +
    " without leading or trailing spaces");

I'm a string without leading or trailing spaces


undefined

Test for presence of a substring using `indexOf()`:

In [6]:
var myString = "foobar";
console.log(myString.indexOf("foo") !== -1); // => true

true


undefined

Generate a string containing the 26-letter sequence of US-ASCII alphabetic characters:

In [63]:
function alphabetString(letterCase) {
    var str  = '',
        charCode = (letterCase === 'upper') ? 65 : 97;
    for (var i = 0; i < 26; ++i) {
        str += String.fromCharCode(charCode + i);
    }
    return str;
}

console.log(alphabetString('lower')); // => abcdefghijklmnopqrstuvwxyz
console.log(alphabetString('upper')); // => ABCDEFGHIJKLMNOPQRSTUVWXYZ

abcdefghijklmnopqrstuvwxyz
ABCDEFGHIJKLMNOPQRSTUVWXYZ


undefined

## Arrays

### Testing arrays

The only reliable way to test for an empty array is to test for its length:

In [2]:
var arr = [];
console.log(!arr.length); // => true

true


undefined

Test for presence of element of an array using `indexOf()`:

In [7]:
console.log([1, 2, 3].indexOf(1) !== -1);         // => true
console.log(["foo", "bar"].indexOf("foo") !== -1) // => true

true
true


undefined

### Creating an array

In [7]:
console.log('foo foo foobarbarous'.match(/\bfoo\b/g));

[ 'foo', 'foo' ]


undefined

### Iterating over an array

One can use `for..in` if **neither the index order nor the length of the array** are meaningful. ES 5 `forEach()` is an alternative to the `for` loop, but a `forEach()` loop cannot be stopped or broken.

Using `for..in`. Note [for...in - JavaScript | MDN](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Statements/for...in):

>  There is no guarantee that for...in will return the indexes in any particular order and it will return all enumerable properties, including those with non–integer names and those that are inherited.
>
> Because the order of iteration is implementation-dependent, iterating over an array may not visit elements in a consistent order. Therefore it is better to use a for loop with a numeric index (or Array.prototype.forEach() or the for...of loop) when iterating over arrays where the order of access is important.

In [8]:
var a = [1, 2, 3];
for (var elt in a) console.log(a[elt]); // 1\n2\n3

1
2
3


undefined

Example of a case where neither index order nor array length are needed: getting the min and max values of an array.

In [11]:
function getArrayMinMax(a) {
    var min = a[0], max = a[0];
    for (var elt in a) {
        min = (a[elt] < min) ? a[elt] : min;
        max = (a[elt] > max) ? a[elt] : max;
    }
    return [min, max];
}

console.log(getArrayMinMax([1])); // => [1, 1]
console.log(getArrayMinMax([1, 1])); // => [1, 1]
console.log(getArrayMinMax([0, 1, 2])); // => [0, 2]
console.log(getArrayMinMax([2, 1, 0])); // => [0, 2]
console.log(getArrayMinMax([2334454, 5])); // => [5, 2334454]

[ 1, 1 ]
[ 1, 1 ]
[ 0, 2 ]
[ 0, 2 ]
[ 5, 2334454 ]


undefined

Another example: testing an array for duplicate values.

In [7]:
function arrayHasDupes(arr) {
    var arrCopy = [];
    for (var elt in arr) {
        if (arrCopy.indexOf(arr[elt]) !== -1) return true;
        arrCopy.push(arr[elt]);
    }
    return false;
}

console.log(arrayHasDupes([0, 1, 2, 3]));          // => false
console.log(arrayHasDupes([0, 1, 1, 3]));          // => true
console.log(arrayHasDupes(['foo', 'bar', 'baz'])); // => false
console.log(arrayHasDupes(['foo', 'bar', 'foo'])); // => true

false
true
false
true


undefined

And another: since strings are arraylike, we can use the same method to determine if a string is an isogram:

In [9]:
function isIsogram(str) {
    if (str === '') return true;
    str = str.toLowerCase();
    var strCopy = '';
    for (var char in str) {
        if (strCopy.indexOf(str[char]) !== -1) return false;
        strCopy += str[char];
    }
    return true;
}

console.log(isIsogram('isogram')); // => true
console.log(isIsogram('abba'));    // => false
console.log(isIsogram('foO'));     // => false

true
false
false


undefined

If array index order and/or array length are meaningful values, use a `for` loop (and initialize the length variable inside the loop's initialization expression, rather than in its condition expression):

In [4]:
var a = [1, 2, 3];
for (var i = 0, l = a.length; i < l; i++) console.log(a[i]); // 1\n2\n3

1
2
3


undefined

Using ES5 `forEach()`:

In [10]:
var a = [1, 2, 3];
a.forEach(function(currVal) {console.log(currVal);}); // 1\n2\n3

1
2
3


undefined

### Sorting an array

Bubble sort:

In [1]:
function bubbleSort(arr) {
    var i, j, elt, next;
    for (i = 0, len = arr.length; i < len; i++) {
        for (j = 0; j < len; j++) {
            elt = arr[j], next = arr[j + 1]; 
            if (next <= elt) arr[j] = next, arr[j + 1] = elt;
        }
    }
    return arr;
}

console.log(bubbleSort([6, 3, 4, 2, 5, 1, 0])); // => [0, 1, 2, 3, 4, 5, 6]

[ 0, 1, 2, 3, 4, 5, 6 ]


undefined

### Transforming an array

Declarative rather than imperative style: use the Array prototype's `map()` method instead of a `for` loop to transform an array.

In [21]:
console.log([1, 2, 3].map(String));       // => ['1', '2', '3']
console.log(['1', '2', '3'].map(Number)); // => [1, 2, 3]

[ '1', '2', '3' ]
[ 1, 2, 3 ]


undefined

In [18]:
function intToArr(int) {
  return int.toString().split('').map(Number);
}

console.log(intToArr(12345)); // => [1, 2, 3, 4, 5]

[ 1, 2, 3, 4, 5 ]


undefined

In [11]:
var myArray = [0, 1, 2, 3, 4];
var myArrayTransformed = myArray.map(function(arrayElement) {
        return arrayElement * 2;
    });

console.log(myArrayTransformed); // => [ 0, 2, 4, 6, 8 ]

[ 0, 2, 4, 6, 8 ]


undefined

In [12]:
function capStrElements(arr) {
    return arr.map(function(currVal) {
        return currVal.charAt(0).toUpperCase() +
               currVal.substring(1).toLowerCase();
    });
}

console.log(capStrElements(['foo', 'bar', 'baz'])); // => ['Foo', 'Bar', 'Baz']
console.log(capStrElements(['FOO', 'BAR', 'BAZ'])); // => ['Foo', 'Bar', 'Baz']

[ 'Foo', 'Bar', 'Baz' ]
[ 'Foo', 'Bar', 'Baz' ]


undefined

### Filtering an array

Declarative rather than imperative style: use the Array prototype's `filter()` method to filter an array.

In [4]:
var myArray = [0, 1, 2, 3, 4];
var myArrayFiltered = myArray.filter(function(arrayElement) {
        return arrayElement % 2 === 0;
    });
console.log(myArrayFiltered); // => [ 0, 2, 4 ]

[ 0, 2, 4 ]


undefined

## Objects

### Test for empty object

In [1]:
Object.keys({}).length === 0; // => true

true

### Extend obj1 with properties of obj2

In [9]:
var obj1 = {
    abstraction: "high"
};

var obj2 = {
    abstraction: "middling",
    concretion:  "low"
};

function extend(obj1, obj2) {
    for (var prop in obj2) {

        // Skip properties already belonging to obj1
        if (prop in obj1) {
            continue;
        } else {
            obj1[prop] = obj2[prop];
        }
    }
}

extend(obj1, obj2);
console.log(obj1);

{ abstraction: 'high', concretion: 'low' }


undefined

## Control structures

Here, `array.length` is called during each iteration. If the array's length is static, this isn't necessary:

In [ ]:
for (var i = 0; i < array.length; i++) {...}

Here, `array.length` is only called once, during the initialization of the loop. Thereafter its value is stored in the initialization variable `l`:

In [ ]:
for (var i = 0, l = array.length; i < l; i++) {...}

## Functions

### Recursion

Formulate a limit for recursion, either by using a `while` loop or a return statement preceding the recursive call (e.g., `if (limitCond === 0) return val;`).

Strips leading space, invoking itself recursively:

In [10]:
function stripLeadingSpace(input) {
    while (input.charAt(0) == " ") {
        input = input.substring(1, input.length);
        stripLeadingSpace(input);
    }
    return input;
}

// => "[foo   ]"
console.log("[" + stripLeadingSpace("   foo   ") +"]");

[foo   ]


undefined

Strips trailing space, invoking itself recursively:

In [11]:
function stripTrailingSpace(input) {
    while (input.charAt(input.length - 1) == " ") {
        input = input.substring(0, input.length - 1);
        stripTrailingSpace(input);
    }
    return input;
}

// => "[   bar]"
console.log("[" + stripTrailingSpace("   bar   ") + "]");

[   bar]


undefined

Getting a factorial using recursion:

In [5]:
function getFactorial(int) {

    /* Needed to handle empty product (factorial of 0 = 1),
       but also serves as limit for recursion */
    if (int === 0) return 1;

    return int * getFactorial(int - 1);
}

console.log(getFactorial(0));  // => 1
console.log(getFactorial(1));  // => 1
console.log(getFactorial(5));  // => 120
console.log(getFactorial(10)); // => 3628800

1
1
120
3628800


undefined

Sum positive integers up to positive integer `int` using recursion:

In [6]:
function addUpTo(int) {
    if (int === 0) return int;
    return int + addUpTo(int - 1);
}

console.log(addUpTo(5));  // => 15
console.log(addUpTo(10)); // => 55

15
55


undefined

Reverse a string using recursion:

In [3]:
function reverseString(str) {
    var l = str.length, out = '';
    while (l > 0) {
        out += str[l - 1];
        l -= 1;
        reverseString(str.substring(0, l));
    }
    return out;
}

console.log(reverseString('foo'));         // => 'oof'
console.log(reverseString('hello world')); // => 'dlrow olleh'
console.log(reverseString('abcd'));        // => 'dcba'
console.log(reverseString('12345'));       // => '54321'

oof
dlrow olleh
dcba
54321


undefined

Add a hyphen between any two odd numerals in a number, ignoring zero:

In [15]:
function insertDash(num) {
    var re = /([13579](?=[13579]))/;
    num = num.toString();
    while (re.test(num)) {
        num = num.replace(re, "$1-");
        insertDash(num);
    }
    return num;
}

// Tests
function terseTest(exp) {
    return exp ? 'success' : 'FAILURE';
}

console.log(
    terseTest(insertDash(333565) === '3-3-3-565'),
    terseTest(insertDash(454703) === '454703'),
    terseTest(insertDash(454793) === '4547-9-3')
);

success success success


undefined

Return true if a string is in an array. Recursive solution uses `Array.prototype.shift()` to remove non-matching elements from the array.

In [ ]:
// Imperative solution using iteration
function find(str, arr) {
    for (var elt in arr) {
        if (arr[elt] === str) return true;
    }
    return false;
}

// Imperative solution using recursion
function find(str, arr) {
    while (arr.length > 0) {
        if (arr[0] === str) return true;
        arr.shift();
        find(str, arr);
    }
    return false;
}

Using recursion inside an IIFE (immediately executed function expression). We name the function (here, `f`) so it can call itself recursively until `val2` is assigned a value unequal to `val1`. Apart from naming the IIFE so it can call itself, there's nothing special about the use of recursion inside an IIFE?

In [4]:
function genInt(max) {
    return Math.floor(Math.random() * max);
}

var val1 = genInt(3); console.log('Value of val1: ' + val1);

/* Use a named IIFE and recursion to ensure a value unequal
   to val1 */
var val2 = (function f() {
    var v = genInt(3); console.log('Value of v:    ' + v);
    return (v !== val1) ? v : f();
})();

console.log('Values of val1, val2: ' + val1 + ', ' + val2);

/* Sample console output
    Value of val1: 1
    Value of v:    1
    Value of v:    0
    Values of val1, val2: 1, 0
*/

Value of val1: 1
Value of v:    1
Value of v:    1
Value of v:    1
Value of v:    0
Values of val1, val2: 1, 0


undefined

Drawing with the mouse: a `mousedown` event listener that adds a `mousemove` event listener to call itself recursively. (To stop drawing on mouseup, set an event listener on `mouseup` to remove the event listener on `mousemove`.)

function startPen(e) {
  var canvas = document.getElementById('canvas'),
      x = e.clientX,
      y = e.clientY,
      pen = createPen();
  pen.style.left = x + 'px';
  pen.style.top  = y + 'px';
  canvas.appendChild(pen);
  canvas.addEventListener('mousemove', startPen);
}

### Function closures

Return a function from another function, and when you call it, the returned function will "remember" the variables that were in its scope when you created it. It has "enclosed" or "closed over" those variables:

In [38]:
// A function that returns a function
function makeConcatter(a, b) {
    return function() {
        return a + b; // Variables closed over
    }
}

// Create a new function using `makeConcatter()`
var foo = makeConcatter('foo', 'bar')

// `foo()` has access to the parameters passed to `makeConcatter()`
console.log(foo()); // => 'foobar'

foobar


undefined

The closure will not include variables that are not referenced by the returned function, however:

In [ ]:
function makeConcatter(a, b) {
    var unincluded;          // Not included in closure
    return function() {
        return a + b;
    }
}

Nor will it include local variables defined inside the returned function:

In [ ]:
function makeConcatter(a, b) {
    var unincludedOuter;     // Not included in closure
    return function() {
        var unincludedInner; // Not included in closure 
        return a + b;
    }
}

If a local variable defined inside the returned function shadows a variable in its scope, that shadow variable will also be excluded from the closure:

In [41]:
function makeConcatter(a, b) {
    var unincludedOuter;     // Not included in closure
    return function() {
        var unincludedInner; // Not included in closure
        var a = 'fur';       // Shadows `a`, so not included in closure 
        return a + b;
    }
}

var foo = makeConcatter('foo', 'bar')
console.log(foo()); // => 'furbar'

furbar


undefined

However, if a variable's value is changed outside the returned function, its value will be changed inside the closure. This is because *a closure is an object, and what is stored in the function returning it is a reference to that object* — so its data can be changed:

In [44]:
function makeConcatter(a, b) {
    b = 'baz';                // Changes value of `b` inside the closure
    return function() {
        var a = 'fur';        // Shadows `a`, so not included in closure 
        return a + b;
    }
}

var foo = makeConcatter('foo', 'bar')
console.log(foo()); // => 'furbaz'

furbaz


undefined

Using a closure for event listeners. You may need to use a closure to successfully pass the value of `foo` to the callback:

In [ ]:
element.addEventListener('click', function() {
    toggleDisplay(foo);
});

### IIFEs (immediately invoked function expressions)

An IIFE (immediately invoked function expression) is a function expression formatted with both enclosing and following parentheses, which ensure that the expression is invoked as soon as it is parsed (rather, for example, than being called later):

In [51]:
// Function declaration
function foo() {
    return 'bar';
}

// We need to call this function to invoke it
console.log(foo()); // => 'bar'

// IIFE 
console.log((function() {
   return 'baz';
})()); // => 'baz'

bar
baz


undefined

### Module Pattern

Using an IIFE to encapsulate functionality. The IIFE's closure ensures that no global variables are set at all, preventing namespace collisions when using multiple libraries:

In [49]:
(function() {
    // all code here is inside the IIFE's closure
   var encapsulatedFoo = 'bar';
})();

console.log(encapsulatedFoo); // => ReferenceError: foo is not defined

ReferenceError: evalmachine.<anonymous>:6
console.log(encapsulatedFoo); // => ReferenceError: foo is not defined
            ^
encapsulatedFoo is not defined

To define a public API (and hide everything else), use an IIFE to return a global object whose methods can then be accessed as properties of the object:

In [50]:
var singleGlobalFoo = (function() {
    return {
        bar: function() { return 'baz' },
        baz: function() { return 'bar' }
    }
})();

console.log(singleGlobalFoo.bar()); // => 'baz'
console.log(singleGlobalFoo.baz()); // => 'bar'

baz
bar


undefined

## DOM manipulation

* Instead of using `window.onload = …`, which can be overwritten by use of the same event handler elsewhere, use `window.AddEventListener('load', function() {…});` or `window.AddEventListener('DOMContentLoaded', function() {…});`.
* Use a document fragment (`var frag = document.createDocumentFragment();`) for efficiency in DOM manipulation: append all children to the fragment, then append the fragment to an existing DOM element. This means only a single manipulation of the DOM, rather than many.
* Get form input element data: `var formInputData = document.forms['myForm'].getElementsByTagName('input');`

## Unit testing

Test for **undefined or undeclared** variable, using ternary conditional syntax (although `typeof` returns 'undefined' for **both *undefined* and *undeclared* variables**, it is the only way to test for an undeclared variable without throwing a `ReferenceError`):

In [3]:
// => 'FAILURE: variable "mySpecialFoo" is undefined or undeclared'
typeof mySpecialFoo !== 'undefined'
  ? console.log('SUCCESS: variable "mySpecialFoo" is not undefined or undeclared')
  : console.log('FAILURE: variable "mySpecialFoo" is undefined or undeclared');

FAILURE: variable "mySpecialFoo" is undefined or undeclared


undefined

Test for empty array:

In [15]:
var myArray = [];

myArray.length !== 0
  ? console.log('SUCCESS: Array `myArray` is not empty')
  : console.log('FAILURE: Array `myArray` is empty');

FAILURE: Array `myArray` is empty


undefined

Test for a prototype property:

In [13]:
var MySpecialFooConstructor = function() {};
MySpecialFooConstructor.prototype.x = 'y';

var mySpecialFoo = new MySpecialFooConstructor;

// => 'FAILURE: ...'
mySpecialFoo.constructor.prototype.hasOwnProperty('z')
  ? console.log('SUCCESS: property "z" exist')
  : console.log('FAILURE: property "z" does not exist');

FAILURE: property "z" does not exist


undefined

### Examples of unit testing

A terse example. Logs one line to the console (useful in browser consoles or SublimeTest's build output console):

In [14]:
var myArray = [1, 2, 3];

// Tests
function test(exp) {
    return + exp ? 'success' : 'FAILURE';
}

console.log('Tests: ' +
    test(typeof myArray !== 'undefined'),
    test(myArray.length !== 0),
    test(myArray[0]     === 0)
); // => success success FAILURE

Tests: success success FAILURE


undefined

A more explicit example:

In [ ]:
typeof Book !== 'undefined'
  ? console.log('SUCCESS: Constructor function `Book()` exists')
  : console.log('FAILURE: Constructor function `Book()` does not exist');

Book.prototype.hasOwnProperty('display')
  ? console.log('SUCCESS: Constructor function `Book()`\'s prototype has method "display"')
  : console.log('FAILURE: Constructor function `Book()`\'s prototype does not have method "display"');

typeof books !== 'undefined'
  ? console.log('SUCCESS: Array `books` exists')
  : console.log('FAILURE: Array `books` does not exist');

books.length !== 0
  ? console.log('SUCCESS: Array `books` contains elements')
  : console.log('FAILURE: Array `books` contains no elements');

books[0].genre == 'mystery'
  ? console.log('SUCCESS: First added object has "genre" property value "mystery"')
  : console.log('FAILURE: First added object does not have "genre" property value "mystery"');

### Unit testing with QUnit

Testing for presence of DOM elements:

In [ ]:
QUnit.test('foo element', function(assert) {
    assert.expect(1);
    assert.ok(document.getElementById('foo'), 'foo element exists');
});

In [ ]:
QUnit.test('fooForm and fooForm input elements', function(assert) {
    assert.expect(2);
    
    // form element
    assert.ok(document.forms.fooForm, 'fooForm element exists');

    // form input element
    assert.ok(document.forms.faveForm.url, 'fooForm input url element exists');

});

Testing form data:

In [ ]:
/* Can we retrieve a data object from the empty (unsubmitted) form in the
format we want? */
QUnit.test('return correct blank form data object', function(assert) {
    assert.expect(1);
    var actual   = getFormInputData(),
        expected = {
            url      : "",
            title    : "",
            comment  : "",
            tags     : ""
        };
    assert.propEqual(actual, expected, 'correct blank form data object returned');
});

Testing a form data validation function (here, `validateFormInputData()`) that returns a Boolean value:

In [ ]:
QUnit.test('validate simulated acceptable form input', function(assert) {
    assert.expect(1);
    var acceptableInput = 'Lorem ipsum dolor sit amet, consectetur adipisicing elit. Minima ' + 
                          'magni reprehenderit tempore, omnis facilis nemo, at repudiandae '  +
                          'animi nihil delectus molestias distinctio quas mollitia, ratione ' +
                          'unde earum. Nihil suscipit, aperiam.';
    var actual   = validateFormInputData(acceptableInput);
    var expected = true;
    assert.strictEqual(actual, expected, 'simulated acceptable form input validated correctly');
});